In [1]:
from dotenv import load_dotenv
import os

In [5]:
load_dotenv()

True

In [11]:
import mysql.connector
import sqlalchemy
import pandas as pd

In [13]:
print("mysql-connector-python version:", mysql.connector.__version__)
print("SQLAlchemy version:", sqlalchemy.__version__)
print("pandas version:", pd.__version__)
print("All libraries installed and imported successfully!")

mysql-connector-python version: 9.2.0
SQLAlchemy version: 2.0.39
pandas version: 2.2.3
All libraries installed and imported successfully!


In [14]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import traceback

In [18]:
DB_USER=os.getenv("DB_USER")
DB_PASS=os.getenv("DB_PASSWORD")
DB_HOST=os.getenv("DB_HOST")
DB_PORT=os.getenv("DB_PORT")
DB_NAME=os.getenv("DB_NAME")

In [23]:
# Create SQLAlchemy engine
try:
    engine = create_engine(
        f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}",
        echo=True, # verbose logging for debugging
        pool_pre_ping=True # Test connections before use
    )
    print("Engine created successfully!")
except Exception as e:
    print("Failed to create engine:")
    print(traceback.format_exc())

Engine created successfully!


In [24]:
# Configure session factory
try:
    Session = sessionmaker(bind=engine)
    session = Session()
    print("Session factory and instance created!")
except Exception as e:
    print("Failed to create session:")
    print(traceback.format_exc())

Session factory and instance created!


In [25]:
# Test connection
try:
    result = session.execute(text("SELECT 1")).fetchall()
    print("Session connection to DB established successfully!")
    print(f"Test query result: {result}") # should print[(1,)]
except Exception as e:
    print("Connection test failed:")
    print(traceback.format_exc())
finally:
    session.close()
    print("Session closed.")

2025-03-22 11:29:41,124 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-03-22 11:29:41,125 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-22 11:29:41,132 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-03-22 11:29:41,133 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-22 11:29:41,138 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-03-22 11:29:41,139 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-22 11:29:41,149 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-22 11:29:41,170 INFO sqlalchemy.engine.Engine SELECT 1
2025-03-22 11:29:41,172 INFO sqlalchemy.engine.Engine [generated in 0.00219s] {}
Session connection to DB established successfully!
Test query result: [(1,)]
2025-03-22 11:29:41,178 INFO sqlalchemy.engine.Engine ROLLBACK
Session closed.


In [28]:
# List databases
session = Session()
try:
    dbs = pd.read_sql(text("SHOW DATABASES"), session.bind)
    print("Databases:", dbs)
    assert DB_NAME in dbs['Database'].values, f"{DB_NAME} not found!"
except Exception as e:
    print("Database check failed:", e)
finally:
    session.close()

2025-03-22 11:33:19,393 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-22 11:33:19,395 INFO sqlalchemy.engine.Engine SHOW DATABASES
2025-03-22 11:33:19,396 INFO sqlalchemy.engine.Engine [generated in 0.00333s] {}
2025-03-22 11:33:19,517 INFO sqlalchemy.engine.Engine ROLLBACK
Databases:              Database
0  information_schema
1    movie_downloader
2  performance_schema


In [29]:
dbs

,Database
0,information_schema
1,movie_downloader
2,performance_schema


In [32]:
# List tables
session = Session()
try:
    tables = pd.read_sql(text("SHOW TABLES"), session.bind)
    print("Tables in", DB_NAME, ":", tables)
    assert not tables.empty, f"No tables found in database {DB_NAME}!"
except Exception as e:
    print("Table check failed:", e)
finally:
    session.close()

2025-03-22 11:36:55,868 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-22 11:36:55,869 INFO sqlalchemy.engine.Engine SHOW TABLES
2025-03-22 11:36:55,870 INFO sqlalchemy.engine.Engine [generated in 0.00270s] {}
2025-03-22 11:36:56,000 INFO sqlalchemy.engine.Engine ROLLBACK
Tables in movie_downloader :   Tables_in_movie_downloader
0              movie_reports
1                      users


In [33]:
tables

,Tables_in_movie_downloader
0,movie_reports
1,users
